In [ ]:
#@title
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Feb  2 13:49:38 2022

@author:
"""


import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow, show, subplot, title, axis, figure



def warpImages(kp, img, M):

    # Get size of the output image

    h=kp[2].pt[1]
    w=kp[2].pt[0]
            
    new_img = cv.warpPerspective(img, M,(np.int32(w),np.int32(h)))

    return new_img


def find_transformation(kpt1, kpt2, matches,transformatioo):
    # Find an Homography matrix between two pictures
    # Transforming keypoints to list of points
   
    dst_pts = np.float32([kpt1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    src_pts = np.float32([kpt2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)


    if transformation=='rigid':
      print(transformation)

      # Compute a rigid transformation (only scale + rotation + translation) 
    
      M, mask = cv.estimateAffinePartial2D(src_pts, dst_pts)
      print('mask : ' , mask)
      print(M)
      
      # Add à third neutral line to make a 3x3 matrix
      affine_row = [0, 0, 1]
      M = np.vstack((M, affine_row))

    elif transformation=='affine':
      print(transformation)

      # Compute affine transformation (6 parameters)

      M, mask = cv.estimateAffine2D(src_pts, dst_pts) 
      print('mask : ' , mask)
      print(M)
      
      # Add à third neutral line to make a 3x3 matrix
      affine_row = [0, 0, 1]
      M = np.vstack((M, affine_row))

    elif transformation=='perspective':
      print(transformation)
      # Compute an homography (8 parameters)
      M, mask = cv.findHomography(src_pts, dst_pts,cv.RANSAC,5)
      print('mask : ' , mask)
      print(M)

    matchesMask = mask.ravel().tolist()
    
    return M,matchesMask


# Read images
img1c = cv.imread('hello.jpg', 1)
img1 = cv.cvtColor(img1c, cv.COLOR_BGR2GRAY)
img1c = cv.cvtColor(img1c, cv.COLOR_BGR2RGB)

# Choix du modèle de transformation : 'rigid','affine' ou 'perspective'

transformation='perspective'

# Données pour l'image redressée au format A4
resol=5
x1=0
y1=0
sizeH=297
sizeV=210
x2=resol*sizeH
y2=resol*sizeV

pts1=[]
pts1.append([x1,y1])
pts1.append([x2,y1])
pts1.append([x2,y2])
pts1.append([x1,y2])

# Deux outliers

pts1.append([x2/2,y2/2])
pts1.append([x1/2,y1/2])


# données pour g4.jpg

x1=2030
y1=110
x2=2750
y2=904
x3=920
y3=1638
x4=363
y4=663


# données pour f4.jpg

x1=244
y1=949
x2=1450
y2=924
x3=1541
y3=2592
x4=225
y4=2621


# données pour hello.jpg

x1=71
y1=238
x2=355
y2=114
x3=471
y3=260
x4=186
y4=438


# Deux outliers

x5=(x1+x2)/2
y5=(y3+y4)/4

x6=(y1+y2)/2
y6=(x2+x4)/3


pts2=[]
pts2.append([x1,y1])
pts2.append([x2,y2])
pts2.append([x3,y3])
pts2.append([x4,y4])
pts2.append([x5,y5])
pts2.append([x6,y6])

kp1 = [cv.KeyPoint(x[0], x[1], 1) for x in pts1]
kp2 = [cv.KeyPoint(x[0], x[1], 1) for x in pts2]

print('points feuille')
for i in range (len(pts1)):
    print(kp1[i].pt)

print('points image')
for i in range (len(pts2)):
    print(kp2[i].pt)


goodMatches=[]
for i in range(len(pts1)):
    m=cv.DMatch(i,i,0)
    goodMatches.append(m)

print('matchess')
for m in goodMatches:
    (x1, y1) = kp1[m.queryIdx].pt
    (x2, y2) = kp2[m.trainIdx].pt
    print ( x1,y1,x2,y2)

kp_img = cv.drawKeypoints(img1, kp2, None, color=(255, 255, 0))
figure(figsize=(10,10),dpi=100)
imshow(kp_img)
show()

#
## find transformation and compute perspective correction
matrix1to2,mask = find_transformation(kp1, kp2,goodMatches,transformation)
img1to2 = warpImages(kp1,img1c, matrix1to2)
#



print('mask :',mask)









Points d'appui utilisés

In [ ]:
print('mask :',mask)


Image redressée

In [ ]:

figure(figsize=(10,10),dpi=100)
title=('image redressée')
imshow(img1to2)
show()

